In [1]:
import numpy as np
import pandas as pd
from tslearn.utils import to_time_series, to_time_series_dataset
from tslearn.neighbors import KNeighborsTimeSeries

In [107]:
nN = 3
init = 10

data = [np.random.rand(10,2) for _ in range(init)]
labels = np.array([list((np.random.randn(1,2)+2)[0]) for _ in range(init)])

In [108]:
labels

array([[1.99088763, 1.16207404],
       [2.04798152, 1.40914503],
       [2.01163066, 2.86785545],
       [2.23389638, 1.65304167],
       [1.92323022, 2.07622326],
       [2.48041188, 0.93185335],
       [2.21019571, 0.97786993],
       [1.85989428, 0.18480523],
       [2.37482813, 1.21235608],
       [0.93202132, 0.56580436]])

Transforming data to tslearn format

In [40]:
time_series = to_time_series_dataset(data)
time_series.shape

(10, 10, 2)

Unknown points

In [172]:
# test = data[0] # same as first known point

test = np.random.rand(10,2)

test_time_series = to_time_series([test])

pred = np.random.randn(1,2)*0.1

print(test_time_series)
print(pred)

[[[0.28766568 0.74972362]
  [0.09265501 0.66445195]
  [0.95137828 0.48255004]
  [0.85371681 0.64787023]
  [0.1547664  0.30347417]
  [0.35675285 0.20136441]
  [0.94817253 0.59363076]
  [0.81994574 0.37974197]
  [0.83295767 0.99526968]
  [0.99352032 0.60333072]]]
[[-0.08453228  0.02624039]]


In [43]:
knn = KNeighborsTimeSeries(n_neighbors=nN).fit(time_series)

In [174]:
labels

array([[1.99088763, 1.16207404],
       [2.04798152, 1.40914503],
       [2.01163066, 2.86785545],
       [2.23389638, 1.65304167],
       [1.92323022, 2.07622326],
       [2.48041188, 0.93185335],
       [2.21019571, 0.97786993],
       [1.85989428, 0.18480523],
       [2.37482813, 1.21235608],
       [0.93202132, 0.56580436]])

In [175]:
def knnUncertainty(nN,pred, test_time_series,labels,knn):

    pred = labels
    sigma = []
    distances, indices = knn.kneighbors(test_time_series,nN, return_distance=True)
    
    for i in range(len(test_time_series)):
        # print(i)
        dist = distances[i]
        print(dist)
        ind = (indices[i])
        print(ind)
        #calculate the nearest point error
        pred_near = pred[i]
        # print(pred_near)
        # print(ind)

        abs_err = np.abs(pred_near - labels[ind])
        print(abs_err)
        weights = 1 - (dist / dist.sum())
        print(weights)
        weighted_err = np.mean(np.average(abs_err, axis=0, weights=weights**nN) )

        nbrs_y = list(labels[ind])
        nbrs_y.append(pred[i])
        nbrs_var = np.std(nbrs_y)

        min_dist = np.min(dist)
        pred_var = weighted_err + min_dist * nbrs_var
        sigma.append(pred_var)
    sigma = np.array(sigma)
    return sigma

In [177]:
knnUncertainty(nN, pred, test_time_series,labels,knn)

[1.18175037 1.48318867 1.49557113]
[6 7 5]
[[0.21930808 0.18420411]
 [0.13099335 0.97726882]
 [0.48952425 0.23022069]]
[0.71596023 0.64350798 0.64053179]


array([1.21628249])

In [166]:
a = np.array([[0,         0],
 [0.05709389, 0.24707099],
 [0.48952425, 0.23022069]])

In [134]:
a

array([[0.        , 0.        ],
       [0.05709389, 0.24707099],
       [0.48952425, 0.23022069]])

In [137]:
weights = np.array([1, 0.51519018, 0.48480982])
weights

array([1.        , 0.51519018, 0.48480982])

In [153]:
weights2 = np.concatenate([weights, weights]).reshape(2,3).transpose()

In [136]:
np.mean(np.average(a, axis=0, weights=weights))

0.1264105435210473

In [155]:
np.average(a, weights=weights2)

0.1264105435210473